In [ ]:
!pip install git+https://github.com/TeamHG-Memex/sklearn-crfsuite.git

  Cloning https://github.com/TeamHG-Memex/sklearn-crfsuite.git to /tmp/pip-req-build-y1fu6aj5
  Running command git clone -q https://github.com/TeamHG-Memex/sklearn-crfsuite.git /tmp/pip-req-build-y1fu6aj5
     |████████████████████████████████| 757kB 2.8MB/s 
  Created wheel for sklearn-crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10667 sha256=3537eafb03afdfdc80c23e6371f7fc86cb0b67dd98269a215f8b3e19ee9e34ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-elmxgk62/wheels/49/ec/e1/9358b7e8b847f19df0c27e92d888ccedc0843e6a154455a6a0
Successfully built sklearn-crfsuite


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ner_dataset.csv', encoding = "ISO-8859-1")

In [ ]:
df = df.fillna(method = 'ffill')

In [ ]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:

#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]


'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
sentences = getter.sentences


In [ ]:

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-art       0.45      0.16      0.23        83
       B-eve       0.43      0.31      0.36        58
       B-geo       0.85      0.91      0.88      7544
       B-gpe       0.97      0.93      0.95      3327
       B-nat       0.75      0.53      0.62        34
       B-org       0.81      0.73      0.77      4032
       B-per       0.85      0.83      0.84      3444
       B-tim       0.93      0.88      0.90      3982
       I-art       0.00      0.00      0.00        65
       I-eve       0.28      0.27      0.27        45
       I-geo       0.82      0.80      0.81      1517
       I-gpe       0.82      0.50      0.62        36
       I-nat       0.75      0.60      0.67         5
       I-org       0.82      0.79      0.81      3356
       I-per       0.84      0.91      0.88      3545
       I-tim       0.86      0.76      0.80      1286
           O       0.99      0.99      0.99    179504

    accuracy              